[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/abigailhaddad/fedscope_new/blob/main/demo.ipynb)

# OPM Federal Workforce Data: Accessions & Separations Over Time

This notebook loads federal workforce accession (new hires) and separation (departures) data from HuggingFace and visualizes trends over time.

**No authentication required** - all datasets are public.

In [ ]:
# Install dependencies (for Colab)
!pip install -q duckdb pandas plotly

In [ ]:
import duckdb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

## 1. Define Dataset URLs

The data is stored as parquet files on HuggingFace. DuckDB can query them directly via HTTP.

In [ ]:
# Generate all dataset URLs
HF_USERNAME = "abigailhaddad"
BASE_URL = f"https://huggingface.co/datasets/{HF_USERNAME}"

def generate_months(start_year=2021, start_month=1, end_year=2025, end_month=11):
    """Generate list of YYYYMM strings."""
    months = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            if year == start_year and month < start_month:
                continue
            if year == end_year and month > end_month:
                break
            months.append(f"{year}{month:02d}")
    return months

months = generate_months()

# Build URLs for each dataset
accession_urls = [f"{BASE_URL}/opm-federal-accessions-{m}/resolve/main/data.parquet" for m in months]
separation_urls = [f"{BASE_URL}/opm-federal-separations-{m}/resolve/main/data.parquet" for m in months]

print(f"Generated {len(accession_urls)} accession URLs")
print(f"Generated {len(separation_urls)} separation URLs")
print(f"\nExample: {accession_urls[0]}")

## 2. Load All Data into DuckDB

Load all parquet files once into DuckDB tables. After this, all queries are instant.

In [ ]:
%%time
# Create a persistent DuckDB connection
db = duckdb.connect()

# Load all accessions into one table
acc_url_list = ", ".join([f"'{url}'" for url in accession_urls])
db.execute(f"""
    CREATE TABLE accessions AS 
    SELECT *, 
           personnel_action_effective_date_yyyymm as month
    FROM read_parquet([{acc_url_list}])
""")

# Load all separations into one table  
sep_url_list = ", ".join([f"'{url}'" for url in separation_urls])
db.execute(f"""
    CREATE TABLE separations AS 
    SELECT *,
           personnel_action_effective_date_yyyymm as month
    FROM read_parquet([{sep_url_list}])
""")

# Check counts
acc_count = db.execute("SELECT SUM(CAST(count AS INTEGER)) FROM accessions").fetchone()[0]
sep_count = db.execute("SELECT SUM(CAST(count AS INTEGER)) FROM separations").fetchone()[0]

print(f"Loaded {acc_count:,} accessions and {sep_count:,} separations")
print(f"All data now in memory - filtering will be instant!")

## 3. Plot Accessions vs Separations Over Time

In [ ]:
# Aggregate by month for the overview chart
acc_df = db.execute("""
    SELECT month, SUM(CAST(count AS INTEGER)) as count
    FROM accessions
    GROUP BY month
    ORDER BY month
""").df()
acc_df['date'] = pd.to_datetime(acc_df['month'], format='%Y%m')

sep_df = db.execute("""
    SELECT month, SUM(CAST(count AS INTEGER)) as count
    FROM separations
    GROUP BY month
    ORDER BY month
""").df()
sep_df['date'] = pd.to_datetime(sep_df['month'], format='%Y%m')

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=acc_df['date'], y=acc_df['count'],
    mode='lines+markers', name='Accessions (New Hires)',
    line=dict(color='#2ecc71', width=2), marker=dict(size=6),
    hovertemplate='%{x|%b %Y}<br>%{y:,.0f} new hires<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=sep_df['date'], y=sep_df['count'],
    mode='lines+markers', name='Separations (Departures)',
    line=dict(color='#e74c3c', width=2), marker=dict(size=6),
    hovertemplate='%{x|%b %Y}<br>%{y:,.0f} departures<extra></extra>'
))

fig.update_layout(
    title='Federal Workforce: Monthly Accessions vs Separations (2021-2025)',
    xaxis_title='Month', yaxis_title='Count',
    hovermode='x unified', template='plotly_white',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    yaxis=dict(tickformat=','), height=500
)
fig.show()

In [ ]:
# Net change (accessions - separations)
merged = acc_df.merge(sep_df, on=['month', 'date'], suffixes=('_acc', '_sep'))
merged['net_change'] = merged['count_acc'] - merged['count_sep']
merged['color'] = merged['net_change'].apply(lambda x: '#2ecc71' if x > 0 else '#e74c3c')

fig = go.Figure()

fig.add_trace(go.Bar(
    x=merged['date'],
    y=merged['net_change'],
    marker_color=merged['color'],
    hovertemplate='%{x|%b %Y}<br>Net: %{y:+,.0f}<extra></extra>'
))

fig.add_hline(y=0, line_width=1, line_color='black')

fig.update_layout(
    title='Federal Workforce: Monthly Net Change (Accessions - Separations)',
    xaxis_title='Month',
    yaxis_title='Net Change',
    template='plotly_white',
    yaxis=dict(tickformat=','),
    height=450,
    showlegend=False
)

fig.show()

print(f"\nTotal accessions: {merged['count_acc'].sum():,}")
print(f"Total separations: {merged['count_sep'].sum():,}")
print(f"Net change: {merged['net_change'].sum():+,}")

## 4. Explore the Data Structure

Let's look at what columns are available in the data.

In [ ]:
# Inspect columns
print("Accessions columns:")
print(db.execute("DESCRIBE accessions").df()['column_name'].tolist())

print("\nSeparations columns:")
print(db.execute("DESCRIBE separations").df()['column_name'].tolist())

In [ ]:
# Show sample data
print("Sample accession records:")
db.execute("SELECT * FROM accessions LIMIT 5").df()

## 5. Data Quality Analysis

Some fields have missing, redacted, or placeholder values that can affect analysis.

In [ ]:
# Analyze data quality across ALL accession data (not just one month)
print("=== DATA QUALITY ISSUES (all accessions 2021-2025) ===\n")

total_people = db.execute("SELECT SUM(CAST(count AS INTEGER)) FROM accessions").fetchone()[0]

checks = {
    'duty_station_state': ['REDACTED', 'NO DATA REPORTED', 'UNSPECIFIED'],
    'education_level': ['UNSPECIFIED', 'NO DATA REPORTED'],
    'stem_occupation': ['UNSPECIFIED', 'ALL OTHER OCCUPATIONS'],
    'supervisory_status': ['ALL OTHER POSITIONS'],
}

for col, bad_values in checks.items():
    conditions = " OR ".join([f"{col} = '{v}'" for v in bad_values])
    
    result = db.execute(f"""
        SELECT {col} as value, SUM(CAST(count AS INTEGER)) as people
        FROM accessions
        WHERE {conditions}
        GROUP BY {col}
        ORDER BY people DESC
    """).df()
    
    if len(result) > 0:
        total_bad = result['people'].sum()
        pct = (total_bad / total_people) * 100
        
        print(f"📍 {col}: {total_bad:,.0f} people ({pct:.1f}%) have unusable values")
        for _, row in result.iterrows():
            print(f"   • '{row['value']}': {row['people']:,.0f}")
        print()

## 6. Interactive Filtering

Filter accessions and separations by agency, state, occupation, etc. and view the trends.

In [ ]:
# Get available filter options from the loaded data
def get_options(col):
    """Get unique values for a column, sorted by frequency."""
    result = db.execute(f"""
        SELECT {col} as value, SUM(CAST(count AS INTEGER)) as n
        FROM accessions
        WHERE {col} IS NOT NULL AND {col} != ''
        GROUP BY {col}
        ORDER BY n DESC
    """).df()
    return ['All'] + result['value'].tolist()

# Cache filter options
filter_options = {
    'agency': get_options('agency'),
    'duty_station_state': get_options('duty_station_state'),
    'occupational_group': get_options('occupational_group'),
    'age_bracket': get_options('age_bracket'),
    'education_level': get_options('education_level'),
}

print("Filter options loaded!")
print(f"  Agencies: {len(filter_options['agency'])-1}")
print(f"  States: {len(filter_options['duty_station_state'])-1}")
print(f"  Occupational Groups: {len(filter_options['occupational_group'])-1}")

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create filter widgets
agency_dropdown = widgets.Dropdown(options=filter_options['agency'], value='All', description='Agency:')
state_dropdown = widgets.Dropdown(options=filter_options['duty_station_state'], value='All', description='State:')
occ_dropdown = widgets.Dropdown(options=filter_options['occupational_group'], value='All', description='Occ Group:')

output = widgets.Output()
download_output = widgets.Output()

def build_where_clause(agency, state, occ_group):
    """Build SQL WHERE clause from filter selections."""
    conditions = []
    if agency != 'All':
        conditions.append(f"agency = '{agency}'")
    if state != 'All':
        conditions.append(f"duty_station_state = '{state}'")
    if occ_group != 'All':
        conditions.append(f"occupational_group = '{occ_group}'")
    return "WHERE " + " AND ".join(conditions) if conditions else ""

def update_chart(change=None):
    """Update the chart based on filter selections - now instant!"""
    with output:
        clear_output(wait=True)
        
        where = build_where_clause(
            agency_dropdown.value, 
            state_dropdown.value, 
            occ_dropdown.value
        )
        
        # Query both tables - instant since data is in memory
        filtered_acc = db.execute(f"""
            SELECT month, SUM(CAST(count AS INTEGER)) as count
            FROM accessions {where}
            GROUP BY month ORDER BY month
        """).df()
        
        filtered_sep = db.execute(f"""
            SELECT month, SUM(CAST(count AS INTEGER)) as count
            FROM separations {where}
            GROUP BY month ORDER BY month
        """).df()
        
        if len(filtered_acc) == 0 and len(filtered_sep) == 0:
            print("No data found for these filters.")
            return
        
        # Add date column
        if len(filtered_acc) > 0:
            filtered_acc['date'] = pd.to_datetime(filtered_acc['month'], format='%Y%m')
        if len(filtered_sep) > 0:
            filtered_sep['date'] = pd.to_datetime(filtered_sep['month'], format='%Y%m')
        
        # Create chart
        fig = go.Figure()
        
        if len(filtered_acc) > 0:
            fig.add_trace(go.Scatter(
                x=filtered_acc['date'], y=filtered_acc['count'],
                mode='lines+markers', name='Accessions',
                line=dict(color='#2ecc71', width=2),
                hovertemplate='%{x|%b %Y}<br>%{y:,.0f}<extra></extra>'
            ))
        
        if len(filtered_sep) > 0:
            fig.add_trace(go.Scatter(
                x=filtered_sep['date'], y=filtered_sep['count'],
                mode='lines+markers', name='Separations',
                line=dict(color='#e74c3c', width=2),
                hovertemplate='%{x|%b %Y}<br>%{y:,.0f}<extra></extra>'
            ))
        
        # Build title
        filters = []
        if agency_dropdown.value != 'All':
            filters.append(agency_dropdown.value[:40])
        if state_dropdown.value != 'All':
            filters.append(state_dropdown.value)
        if occ_dropdown.value != 'All':
            filters.append(occ_dropdown.value[:30])
        
        title = "Filtered: " + ", ".join(filters) if filters else "All Federal Workforce"
        
        fig.update_layout(
            title=title,
            xaxis_title='Month', yaxis_title='Count',
            template='plotly_white', height=450,
            yaxis=dict(tickformat=','),
            hovermode='x unified'
        )
        fig.show()
        
        # Show totals
        acc_total = filtered_acc['count'].sum() if len(filtered_acc) > 0 else 0
        sep_total = filtered_sep['count'].sum() if len(filtered_sep) > 0 else 0
        print(f"\nTotal accessions: {acc_total:,.0f}")
        print(f"Total separations: {sep_total:,.0f}")
        print(f"Net change: {acc_total - sep_total:+,.0f}")
    
    # Update download data
    with download_output:
        clear_output(wait=True)
        if len(filtered_acc) > 0 or len(filtered_sep) > 0:
            merged = pd.merge(
                filtered_acc[['month', 'count']].rename(columns={'count': 'accessions'}),
                filtered_sep[['month', 'count']].rename(columns={'count': 'separations'}),
                on='month', how='outer'
            ).fillna(0).sort_values('month')
            merged['net_change'] = merged['accessions'] - merged['separations']
            print("📥 Download data (right-click table → Copy/Save):")
            display(merged)

# Connect widgets
agency_dropdown.observe(update_chart, names='value')
state_dropdown.observe(update_chart, names='value')
occ_dropdown.observe(update_chart, names='value')

# Display
print("Select filters - updates are instant!")
display(widgets.HBox([agency_dropdown, state_dropdown, occ_dropdown]))
display(output)
display(download_output)

update_chart()